In [1]:
import scipy.io
import numpy as np

# Load the .mat file
file_path = './EV_Rank_1_52_RBs_50_UEs_1000_snaps.mat'
data = scipy.io.loadmat(file_path)

# Extract the relevant data
EV_data = data['EV_re_im_split']

# Check the shape and structure of the extracted data
data = EV_data
del EV_data
print(data.shape)

# Function to create sequences
def create_sequences(data, timesteps_in):
    X, y = [], []
    for ue in data:
        for i in range(len(ue) - timesteps_in):
            X.append(ue[i:i + timesteps_in])
            y.append(ue[i + timesteps_in])
    return np.array(X), np.array(y)

timesteps_in = 5

X, y = create_sequences(data, timesteps_in)

# Check the shapes of the result
print(f'X shape: {X.shape}, y shape: {y.shape}')
# Expected X shape: (num_samples, 5, 832), y shape: (num_samples, 832)


(50, 1000, 832)
X shape: (49750, 5, 832), y shape: (49750, 832)


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the TCN model manually using Conv1D layers with adjusted dilation rates
# model = Sequential([
#     Conv1D(filters=64, kernel_size=2, dilation_rate=1, activation='relu', input_shape=(timesteps_in, X.shape[2])),
#     Conv1D(filters=64, kernel_size=2, dilation_rate=1, activation='relu'),
#     Conv1D(filters=64, kernel_size=2, dilation_rate=1, activation='relu'),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dense(X.shape[2])
# ])


# Define the TCN model manually using Conv1D layers with adjusted dilation rates and added dropout layers
model = Sequential([
    Conv1D(filters=128, kernel_size=2, dilation_rate=1, activation='relu', input_shape=(timesteps_in, X.shape[2])),
    Dropout(0.2),
    Conv1D(filters=128, kernel_size=2, dilation_rate=1, activation='relu'),
    Dropout(0.2),
    Conv1D(filters=128, kernel_size=2, dilation_rate=1, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(X.shape[2])
])

model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions
predictions = model.predict(X_test)
print(predictions)


2024-05-22 21:53:56.322352: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/rithvik/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-05-22 21:53:58.954385: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/10


I0000 00:00:1716395042.196891   23966 service.cc:145] XLA service 0x742018004110 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1716395042.196925   23966 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-05-22 21:54:02.234786: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-22 21:54:02.426989: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


  69/2488 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0157  

I0000 00:00:1716395048.216711   23966 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2488/2488 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - loss: 0.0132 - val_loss: 0.0105
Epoch 2/10
2488/2488 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0109 - val_loss: 0.0100
Epoch 3/10
2488/2488 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0106 - val_loss: 0.0098
Epoch 4/10
2488/2488 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0104 - val_loss: 0.0097
Epoch 5/10
2488/2488 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0103 - val_loss: 0.0096
Epoch 6/10
2488/2488 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0103 - val_loss: 0.0096
Epoch 7/10
2488/2488 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0102 - val_loss: 0.0095
Epoch 8/10
2488/2488 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0102 - val_loss: 0.0095
Epoch 9/10
2488/2488 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0101 - val_loss: 0.0094
Epoch 10/10
2488/2488 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0101 - val_loss: 0.0094
311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0094
Test Loss: 0.009424958378076553
311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Make predictions
predictions = model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R^2): {r2}')


311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Mean Squared Error (MSE): 0.009424955178970948
Mean Absolute Error (MAE): 0.07353682222708315
R-squared (R^2): 0.3843458171104089
